In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from util_io import (
    init, finalize, dump_conf, assigned_day_to_family_on_day, assigned_day_to_occupancy
)
from util_cost import (
    cal_total, n_people, family_id_choice_to_pref_cost, cal_total_preference, cal_total_accounting,
    nd_ndp1_to_account_penality, fixed_family_cost, fixed_person_cost
)
from util_cost import choices as family_pref
from util_check import deep_check, check_valid_all

In [2]:
# constants #
N_families = 5000
N_days = 100
N_min_people = 125
N_max_people = 300
# constants #

# params #
#path_init_conf =     '../output/m08-improved-test.csv'
#path_init_conf =     '../input/another_pytorch_implementation.csv'
#path_dump_improved = '../output/m14-improved-v2.csv' # output solution

num_cpu_cores = 6
#time_limit = -1 # unlimited
time_limit = 3*60*60*1000  # in ms

# occupancy_diff = 2  # +- the occupancy of input solution for each day
max_family_rank = 3  # maximum number of rank of the preference days for each family
# use_hint = True      # use current input as hint
# occupancy_count_as_variables = False  # use occupancy_counts as variable
# min_choice_0_families = 3000   # minimum number of families that are at their choice 0
# target_pref_cost = 62868
target_accounting_cost = 6020.043432
target_accounting_cost_error = 0.0001

## Ortools - CBC MIP solver

In [3]:
from ortools.linear_solver import pywraplp

In [4]:
solver = pywraplp.Solver('', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [5]:
if num_cpu_cores > 0:
    print('Set num threads:', solver.SetNumThreads(num_cpu_cores))
if time_limit > 0:
    print('Set time limit:', solver.SetTimeLimit(time_limit))

Set num threads: True
Set time limit: None


In [6]:
allowed_occupancy = range(N_min_people, N_max_people+1)

In [7]:
total_people = n_people.sum()
print(total_people)

21003


In [8]:
# Occupancy pairs counts [N_d, N_d+1]: n of days at that pair
occupancy_pairs_counts = {
    (o, o_next): solver.IntVar(0, N_days, 'o[%i,%i]' % (o, o_next))
    for o in allowed_occupancy for o_next in allowed_occupancy
}

In [9]:
accounting_cost = solver.Sum([
    occupancy_pairs_counts[o, o_next] * nd_ndp1_to_account_penality[o, o_next]
    for o in allowed_occupancy for o_next in allowed_occupancy
])

In [10]:
#accounting_cost.GetCoeffs()

In [11]:
# at least 1 day with o == o_next
solver.Add(solver.Sum([occupancy_pairs_counts[o, o] for o in allowed_occupancy]) >= 1)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x11dea9900> >

In [12]:
# first day is N_max_people = 300
solver.Add(solver.Sum([occupancy_pairs_counts[N_max_people, o] for o in allowed_occupancy]) >= 1)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x11dea9a80> >

In [13]:
# occupancy counts from outgoing
occupancy_outgoing_counts = {
    # outgoing of the o
    o: solver.Sum([occupancy_pairs_counts[o, o_other] for o_other in allowed_occupancy])    
    for o in allowed_occupancy
}
# occupancy counts from incoming
occupancy_incoming_counts = {
    # incoming of the o
    o: solver.Sum([occupancy_pairs_counts[o_other, o] for o_other in allowed_occupancy])
    - (1 if o == N_min_people else 0) + (1 if o == N_max_people else 0)
    for o in allowed_occupancy
}

In [14]:
# o and o_next consistent (assume first day is 300 last day is 125)
for o in allowed_occupancy:
    solver.Add(
        occupancy_outgoing_counts[o] == occupancy_incoming_counts[o]
    )

In [15]:
# sum occupancy == total_people
solver.Add(
    solver.Sum([occupancy_outgoing_counts[o] * o for o in allowed_occupancy]) == total_people
)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x11c0a1bd0> >

In [16]:
# constraint targe accounting cost
solver.Add(accounting_cost <= target_accounting_cost + target_accounting_cost_error)
solver.Add(accounting_cost >= target_accounting_cost - target_accounting_cost_error)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x11b0bfa80> >

In [17]:
solver.Minimize(accounting_cost)

In [18]:
solver.NumVariables()

30976

In [19]:
solver.NumConstraints()

181

### Solve

In [20]:
%%time
# Solve
sol = solver.Solve()

resdict = {0:'OPTIMAL', 1:'FEASIBLE', 2:'INFEASIBLE', 3:'UNBOUNDED', 
           4:'ABNORMAL', 5:'MODEL_INVALID', 6:'NOT_SOLVED'}
print('Result: ', resdict[sol])
print('Total cost = ', solver.Objective().Value())
print("Time = ", solver.WallTime(), " milliseconds")

Result:  NOT_SOLVED
Total cost =  0.0
Time =  1977817  milliseconds
CPU times: user 3h 3s, sys: 5min 57s, total: 3h 6min
Wall time: 32min 56s


In [ ]:
solver.VerifySolution(0.00001, False)

In [ ]:
occupancy_pairs_counts_sol = {
    (o, o_next): occupancy_pairs_counts[o, o_next].solution_value()
    for o in allowed_occupancy for o_next in allowed_occupancy
}

In [ ]:
result_cost = sum([
    occupancy_pairs_counts_sol[o, o_next] * nd_ndp1_to_account_penality[o, o_next]
    for o in allowed_occupancy for o_next in allowed_occupancy
])
is_in_range = abs(result_cost - target_accounting_cost) <= target_accounting_cost_error
print(result_cost, target_accounting_cost, '+-', target_accounting_cost_error)
print('Is in range:', is_in_range)

In [ ]:
occupancy_pairs_counts_sol_arr = np.array([
    [occupancy_pairs_counts_sol[o, o_next] for o_next in allowed_occupancy] for o in allowed_occupancy
])

In [ ]:
plt.imshow(occupancy_pairs_counts_sol_arr, cmap='hot', interpolation='nearest')

In [ ]:
print({i: v for i, v in occupancy_pairs_counts_sol.items() if v > 0})

In [ ]:
out_going_occupancy_day_counts_sol = pd.Series({
    o: sum([occupancy_pairs_counts_sol[o, o_next] for o_next in allowed_occupancy])
    for o in allowed_occupancy
})
in_coming_occupancy_day_counts_sol = pd.Series({
    o_next: sum([occupancy_pairs_counts_sol[o, o_next] for o in allowed_occupancy])
    for o_next in allowed_occupancy
})

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(in_coming_occupancy_day_counts_sol, alpha=0.7, label='in coming')
plt.plot(out_going_occupancy_day_counts_sol, alpha=0.7, label='out going')
plt.legend(); plt.grid(); plt.show()

In [ ]:
plt.figure(figsize=(12, 2))
plt.plot(in_coming_occupancy_day_counts_sol - out_going_occupancy_day_counts_sol)
plt.grid(); plt.show()